In [9]:
import torch
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image

In [13]:
# Paths
model_save_path = "efficientnet_b2_emotion_model.pth"  # Path to the saved model

# Emotion categories
emotion_classes = ["Anger", "Contempt", "Disgust", "Fear", "Happy", "Neutral", "Sad", "Surprise"]

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
# Define the model
def load_model(model_path):
    model = models.efficientnet_b2(pretrained=False)
    model.features[0][0] = nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1, bias=False)  # Adjust for grayscale
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.6),  # Match the dropout used during training
        nn.Linear(model.classifier[1].in_features, len(emotion_classes)),
    )
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = model.to(device)
    model.eval()
    return model

In [15]:
# Load the trained model
model = load_model(model_save_path)

C:\Users\Tuf\AppData\Local\Temp\ipykernel_19148\4189581435.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=devi

In [16]:
# Image preprocessing function
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),  # Ensure grayscale
        transforms.Resize((224, 224)),               # Resize to model input size
        transforms.ToTensor(),                       # Convert to tensor
        transforms.Normalize(mean=[0.485], std=[0.229]),  # Normalize for grayscale images
    ])
    image = Image.open(image_path)
    
    # Check if the image is already grayscale
    if image.mode != "L":  # "L" means grayscale in PIL
        print("Converting image to grayscale...")
        image = image.convert("L")  # Convert to grayscale
    
    image = transform(image)
    image = image.unsqueeze(0)  # Add batch dimension
    return image.to(device)

In [17]:
# Prediction function
def predict_emotion(image_path):
    # Preprocess the input image
    image = preprocess_image(image_path)
    
    # Perform prediction
    with torch.no_grad():
        outputs = model(image)
        probabilities = torch.softmax(outputs, dim=1)
        predicted_class_idx = torch.argmax(probabilities, dim=1).item()
        predicted_class = emotion_classes[predicted_class_idx]
        confidence = probabilities[0, predicted_class_idx].item()

    return predicted_class, confidence

In [18]:
# Example usage
if __name__ == "__main__":
    image_path = "../face_detect_test/tc2.jpg"  # Replace with the path to your image
    predicted_emotion, confidence = predict_emotion(image_path)
    print(f"Predicted Emotion: {predicted_emotion} (Confidence: {confidence:.2f})")

Converting image to grayscale...
Predicted Emotion: Surprise (Confidence: 0.67)
